In [ ]:
import pandas as pd
df_apart= pd.read_csv("../Data/apartments.csv")
#df_apart.head(10)
print("Apartments.com columns:")
print(df_apart.columns.tolist())

In [ ]:
import pandas as pd
df_pad= pd.read_csv("../Data/padmapper.csv")
#df_pad.head(10)
print("\nPadmapper.com columns:")
print(df_pad.columns.tolist())

In [ ]:
df_pad_renamed = df_pad.rename(columns={'address': 'Title', 'price': 'Price', 'bedrooms':'Beds', 'area':'Address', 'url':'URL'})

print(df_pad_renamed.columns.tolist())

In [ ]:
df_pad_renamed.info()
df_apart.info()

In [ ]:
import numpy as np
import pandas as pd

def split_range_column(series, sep='-', is_price=True):
    result = []
    for val in series:
        s = str(val).strip().lower()
        if is_price:
            s = s.replace('$','').replace(',','')
        s = s.replace('beds','').replace('bed','').replace('studio','0').replace(' ', '')
        
        # Split on en dash or hyphen
        if '–' in s:
            left, right = s.split('–',1)
        elif sep in s:
            left, right = s.split(sep,1)
        else:
            left = right = s
        
        try:
            min_val = float(left)
            max_val = float(right)
        except:
            min_val = max_val = np.nan
        
        result.append((min_val, max_val))
    
    arr = np.array(result)
    return pd.Series(arr[:,0]), pd.Series(arr[:,1])

# ----------------------------
# Apply split to both datasets
# ----------------------------
for df in [df_apart, df_pad_renamed]:
    df['price_min'], df['price_max'] = split_range_column(df['Price'], is_price=True)
    df['beds_min'], df['beds_max'] = split_range_column(df['Beds'], is_price=False)


from IPython.display import display

display(df_apart.head(10))      # Show apartments.com data
print("\n--- Padmapper ---\n")
display(df_pad_renamed.head(10))   # Show padmapper.com data


In [ ]:
# ----------------------------
# Fill missing values
# ----------------------------
# Apartments.com → fill based on beds
def fill_rent_based_on_beds(row):
    beds = row['beds_min']
    if beds == 1:
        return 1220
    elif beds == 2:
        return 1460
    else:
        return 1375

for col in ['price_min','price_max']:
    df_apart[col] = df_apart[col].fillna(df_apart.apply(fill_rent_based_on_beds, axis=1))

df_apart['beds_min'] = df_apart['beds_min'].fillna(1)
df_apart['beds_max'] = df_apart['beds_max'].fillna(df_apart['beds_min'])

# Padmapper → fill missing prices with median
median_price_min = df_pad_renamed['price_min'].median()
median_price_max = df_pad_renamed['price_max'].median()
df_pad_renamed['price_min'] = df_pad_renamed['price_min'].fillna(median_price_min)
df_pad_renamed['price_max'] = df_pad_renamed['price_max'].fillna(median_price_max)

df_pad_renamed['beds_min'] = df_pad_renamed['beds_min'].fillna(1)
df_pad_renamed['beds_max'] = df_pad_renamed['beds_max'].fillna(df_pad_renamed['beds_min'])


from IPython.display import display

display(df_apart.head(10))      # Show apartments.com data
print("\n--- Padmapper ---\n")
display(df_pad_renamed.head(10))   # Show padmapper.com data


In [ ]:
# ----------------------------
# Combine datasets
# ----------------------------
import pandas as pd

# Adding a column to track source
df_apart['Source'] = 'apartments.com'
df_pad_renamed['Source'] = 'padmapper.com'

# Merge the datasets (stack them vertically)
df_merged = pd.concat([df_apart, df_pad_renamed], ignore_index=True)

# Reorder columns if you like
cols_order = ['Source', 'Title','Address','URL','Price','Beds','price_min','price_max','beds_min','beds_max']
df_merged = df_merged[cols_order]

# Save to CSV
df_merged.to_csv("../Data/merged.csv", index=False)

# Check
print(df_merged.head(10))
print("\nMissing values check:")
print(df_merged.isnull().sum())


In [ ]:
print(df_merged.head(10))